# Debugging

Other things to maybe look into:
* https://github.com/gruns/icecream
* https://github.com/cool-RR/pysnooper

## Postmortem debugging

I wrote this great code! Unfortunately it is going to crash (there will be an unhandled exception)...

But afterwards I can inspect the stack at the time using `pdb.pm` (pdb postmortem).

When there is an unhandled exception, Python stores a bunch of information in [sys.last_traceback](https://docs.python.org/3.2/library/sys.html#sys.last_traceback). This is where `pdb.pm` pulls the info from.

In [1]:
a = 2
b = [1,2,3]
a + b

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [2]:
import pdb; pdb.pm()

> <ipython-input-1-0eae50b1b6cc>(3)<module>()
-> a + b
(Pdb) print(a)
2
(Pdb) print(b)
[1, 2, 3]
(Pdb) exit


## Auto postmortem debugging

If you always want to get dropped into a debugger on an error you can set that up with the pdb magic. I find this a little bit annoying (often the error is obvious and I don't need the debugger and typing `exit` is so hard...) so tend to leave it off.

In [1]:
%pdb 1
a + b

Automatic pdb calling has been turned ON


NameError: name 'a' is not defined

> <ipython-input-1-89736cf0aa47>(2)<module>()
      1 get_ipython().run_line_magic('pdb', '1')
----> 2 a + b

ipdb> exit


In [4]:
%pdb 0
# Leave this turned off

Automatic pdb calling has been turned OFF


## Premortem 

Sometimes you are writing code that you just want to check really thoroughly to make sure everything is happening as you imagined. Premortem debugging (stepping though) is useful then.

Slightly weird things happen if you put the `pdb.set_trace` outside of a function. The `n` steps into `IPython.core` code which you probably don't want. See [here](https://stackoverflow.com/questions/46495269/debuggers-not-acting-properly-on-jupyter-notebooks).

If you do want to start traces outside of functions (a totally reasonable thing!) I would suggest just setting a breakpoint on the next line.

In [1]:
def binarize(x, threshold):
    if x > threshold:
        return 1
    else:
        return 0

# Ahh this is to complicated for me to think all the way through
import pdb;pdb.set_trace()
binarize(2, 1)


--Return--
> <ipython-input-1-a87650b633d8>(8)<module>()->None
-> import pdb;pdb.set_trace()
(Pdb) l
  3  	        return 1
  4  	    else:
  5  	        return 0
  6  	
  7  	# Ahh this is to complicated for me to think all the way through
  8  ->	import pdb;pdb.set_trace()
  9  	binarize(2, 1)
[EOF]
(Pdb) b 9
Breakpoint 1 at <ipython-input-1-a87650b633d8>:9
(Pdb) c
> <ipython-input-1-a87650b633d8>(9)<module>()->None
-> binarize(2, 1)
(Pdb) s
--Call--
> <ipython-input-1-a87650b633d8>(1)binarize()
-> def binarize(x, threshold):
(Pdb) print("etc etc")
etc etc
(Pdb) exit


BdbQuit: 

In Python 3.7, this is even easier with the [breakpoink](https://www.python.org/dev/peps/pep-0553/) builtin. You can just replace `import pdb; pdb.set_trace()` with this. E.g.

In [5]:
def f(a, b):
    breakpoint()
    return a + b

f(1, 3)

> <ipython-input-5-06c5769ebd16>(3)f()
-> return a + b
(Pdb) c


4

## Better print debugging

### Icecream

You are (I am) going to start off doing print debugging. I don't think this is too bad. However there are some things that can make print debugging better.

In [14]:
from icecream import ic

Inspects the line + values

In [13]:
a, b = 2, "hello"

def f(a):
    return a + a
ic(2 + f(a))
ic("two" + f(b))

ic| 2 + f(a): 6
ic| "two" + f(b): 'twohellohello'


'twohellohello'

Can tell you what was executed.

In [15]:
a = 17
if a > 20:
    ic()
    # do some stuff
else:
    ic()
    # do some other stuff

ic| <ipython-input-15-06f16a680673>:6 in <module> at 20:15:13.766


### Pysnooper

Seems kinda like the result of stepping through code.

In [25]:
import pysnooper

In [33]:
# Could also just wrap a piece of code with `with pysnooper.snoop():`
@pysnooper.snoop()
def num_to_binary(x):
    s = 1
    while s <= x/2:
        s = s*2
    b = ""
    while s >= 1:
        if x >= s:
            b += "1"
            x -= s
        else:
            b += "0"
        s /= 2
    return b
num_to_binary(10)

Starting var:.. x = 10
16:29:19.741592 call         3 def num_to_binary(x):
16:29:19.741718 line         4     s = 1
New var:....... s = 1
16:29:19.741775 line         5     while s <= x/2:
16:29:19.741813 line         6         s = s*2
Modified var:.. s = 2
16:29:19.741860 line         5     while s <= x/2:
16:29:19.741893 line         6         s = s*2
Modified var:.. s = 4
16:29:19.741938 line         5     while s <= x/2:
16:29:19.741969 line         6         s = s*2
Modified var:.. s = 8
16:29:19.742013 line         5     while s <= x/2:
16:29:19.742044 line         7     b = ""
New var:....... b = ''
16:29:19.742087 line         8     while s >= 1:
16:29:19.742119 line         9         if x >= s:
16:29:19.742150 line        10             b += "1"
Modified var:.. b = '1'
16:29:19.742192 line        11             x -= s
Modified var:.. x = 2
16:29:19.742235 line        14         s /= 2
Modified var:.. s = 4.0
16:29:19.742282 line         8     while s >= 1:
16:29:19.742313 lin

'1010'